https://towardsdatascience.com/multivariate-linear-regression-in-python-step-by-step-128c2b127171

https://www.mrlc.gov/data/legends/national-land-cover-database-2016-nlcd2016-legend

https://medium.com/analytics-vidhya/implementing-linear-regression-using-sklearn-76264a3c073c


In [1]:
# !pip install scikit-learn

In [2]:
import pandas as pd
import numpy as np
import time
from sklearn.linear_model import LinearRegression
pd.set_option('display.max_columns', None)

In [3]:
catch_file = './Catchment Data/Catchment_Regression_Data.csv'
catch_data = pd.read_csv(catch_file)
catch_data

,CatchId,SLOPE,LENGTHKM,AREASQKM,StreamOrde,Avg_Lon,Avg_Lat,Avg_Elev,RunoffC,MajLC,Roughness
0,932020122,0.049157,4.754,6.0120,1.0,-75.185438,43.127256,205.3807,NaN,NaN,0.050000
1,3246072,0.004730,2.260,1.6632,4.0,-74.565526,42.681145,371.1816,45.21915,81.0,0.005000
2,3246388,0.000030,0.168,0.0378,3.0,-74.156619,42.612042,285.9409,43.21429,81.0,0.005000
3,3247620,0.005577,0.624,1.0278,1.0,-74.116866,42.232266,716.3624,45.54291,43.0,0.142483
4,3247570,0.009604,21.108,80.7237,1.0,-74.214211,42.256132,650.3924,41.60782,41.0,0.228417
...,...,...,...,...,...,...,...,...,...,...,...
43279,10466733,0.000030,0.336,0.1845,NaN,NaN,NaN,NaN,NaN,NaN,0.050000
43280,10466483,0.000030,0.378,0.3438,NaN,NaN,NaN,NaN,NaN,NaN,0.050000
43281,10469587,0.002440,1.168,0.9054,NaN,NaN,NaN,NaN,NaN,NaN,0.050000
43282,10469373,0.000551,0.363,0.3555,NaN,NaN,NaN,NaN,NaN,NaN,0.050000


In [4]:
catch_data.columns

Index(['CatchId', 'SLOPE', 'LENGTHKM', 'AREASQKM', 'StreamOrde', 'Avg_Lon',
       'Avg_Lat', 'Avg_Elev', 'RunoffC', 'MajLC', 'Roughness'],
      dtype='object')

In [5]:
# Add corrections determined in Notebook 01.2
catch_data.loc[catch_data.CatchId == 6251308, 'SLOPE'] = 0.15 / 5223 # PASS1
catch_data.loc[catch_data.CatchId == 6251234, 'SLOPE'] = 0.36 / 6934 # PASS2 
catch_data.loc[catch_data.CatchId == 6251128, 'SLOPE'] = 1.27 / 1729 # PASS3
catch_data.loc[catch_data.CatchId == 6249784, 'SLOPE'] = 0.00005 # PASS7 - arbitrary because slope is almost zero
catch_data.loc[catch_data.CatchId == 6251122, 'SLOPE'] = 0.37 / (694 + 701) # PASS10
catch_data.loc[catch_data.CatchId == 6251144, 'SLOPE'] = 0.37 / (694 + 701) # PASS10
catch_data.loc[catch_data.CatchId == 6250358, 'SLOPE'] = 0.16 / 2744 # PASS16
catch_data.loc[catch_data.CatchId == 6250388, 'SLOPE'] = 0.71 / 3277 # PASS17
catch_data.loc[catch_data.CatchId == 6250418, 'SLOPE'] = 1.40 / 2824 # PASS18

catch_data.loc[catch_data.CatchId == 6246206, 'SLOPE'] = 1.59 / 2824 # WANQ2
catch_data.loc[catch_data.CatchId == 6250622, 'SLOPE'] = 1.41 / 2480 # POMP4/5
catch_data.loc[catch_data.CatchId == 6249738, 'SLOPE'] = 0.00005     # POMP2 - arbitrary value to remove error
# catch_data[catch_data.CatchId == 6246098, 'SLOPE'] = 0.88 / 1013 # RAMA5 - 
catch_data.loc[catch_data.CatchId == 932030072, 'SLOPE']=1.75 / 635  # RAMA5
catch_data.loc[catch_data.CatchId == 6250680, 'SLOPE'] = 0.00005     # ROCK2 - arbitrary value to remove error

# 020402
catch_data.loc[catch_data.CatchId == 4491888, 'SLOPE'] = 0.5 / 3571 # NBRAN4

catch_data.loc[catch_data.SLOPE < 0.00003, 'SLOPE'] = 0.00003

In [6]:
file = './HWM/Iter3/HWM_Combined_Opt.csv'
# print(file[:30])
rough_data = pd.read_csv(file)
rough_data = rough_data.drop_duplicates(subset='COMID')
rough_data = rough_data.dropna(subset=['COMID'])
rough_data

,Unnamed: 0,Unnamed: 0.1,SITE_NUM,HWM_ID,SITE_ID,NAME,LAT_DD,LON_DD,ELEV_NAVD88_ft,ELEV_NAVD88_m,DEM_m,HAND_m,COMID,Inun_m,Stream_H_m,Cleaning_note,Error_note,fmax,time_file,roughness,rmse,Predict_Stream_H_m
7,7,7,NaN,NaN,Somerset,38,40.554048,-74.583915,47.4109,14.450842,10.978020,3.150568,9513852.0,3.472823,6.623391,9513368 to 9513852,Should be comid 9513852 20110828_18 1597.179964,1597.179964,20110828_18,0.005874,4.564539e-06,6.623395
8,8,8,145.0,1.0,WANQ2,WANQ2_1,41.006944,-74.292556,197.7310,60.268409,58.991779,0.700825,6246206.0,1.276630,1.977454,slope recalc,Downstream of the Wanaque reservoir - human co...,561.659987,20110828_15,0.005000,8.580067e-01,2.667674
17,17,17,56.0,2.0,PASS3,PASS3_2,40.864833,-74.109389,14.9250,4.549140,3.451653,3.451653,6251128.0,1.097487,4.549140,slope revise,"tidal? Catchment has minimum slope of 0.00001,...",2489.409944,20110829_04,0.008737,5.713186e-02,4.563997
24,24,24,55.0,1.0,PASS2,PASS2_1,40.822222,-74.123278,11.2725,3.435858,2.726151,2.726151,6251234.0,0.709707,3.435858,slope revise,"tidal? Catchment has minimum slope of 0.00001,...",2472.239945,20110829_06,0.005000,2.232903e+00,5.806001
31,31,31,54.0,1.0,PASS1,PASS1_1,40.786056,-74.148361,9.5150,2.900172,2.406661,2.406661,6251308.0,0.493511,2.900172,slope revise,"tidal? Catchment has minimum slope of 0.00001,...",2449.529945,20110829_08,0.005000,2.511118e+00,5.654670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997,997,997,NaN,NaN,Somerset,41,40.599510,-74.672659,75.3170,22.956622,22.792509,3.649328,9513148.0,0.164112,3.813441,NaN,NaN,782.619982,20110828_14,0.076077,1.651314e-07,3.813441
998,998,998,NaN,NaN,Somerset,18,40.500766,-74.584019,44.9056,13.687227,13.645301,4.672276,9513658.0,0.041926,4.714202,NaN,NaN,987.729978,20110828_16,0.196399,1.168382e-07,4.714202
999,999,999,NaN,NaN,Somerset,61,40.585048,-74.508256,38.7454,11.809598,11.718499,3.299677,9513328.0,0.091099,3.390776,NaN,NaN,213.399995,20110828_07,0.064588,1.166406e-07,3.390776
1000,1000,1000,NaN,NaN,Somerset,49,40.468158,-74.648728,71.0831,21.666129,20.009172,0.427351,9514754.0,1.656956,2.084307,NaN,NaN,44.249999,20110828_05,0.062688,1.078111e-07,2.084308


In [7]:
data = catch_data.merge(rough_data[['COMID','roughness']], left_on='CatchId', right_on='COMID')
data = data.drop(columns=['CatchId','COMID'])
data

,SLOPE,LENGTHKM,AREASQKM,StreamOrde,Avg_Lon,Avg_Lat,Avg_Elev,RunoffC,MajLC,Roughness,roughness
0,0.000612,5.019,8.6067,4.0,-74.102551,40.936324,20.043160,62.65450,22.0,0.027188,0.027188
1,0.007341,0.331,0.9576,6.0,-74.180779,40.922627,40.657850,77.03477,23.0,0.022162,0.022162
2,0.002241,1.352,1.0440,6.0,-74.221956,40.890790,48.535850,71.74138,23.0,0.010536,0.010536
3,0.000030,0.985,0.8874,4.0,-74.078219,40.902936,14.687020,67.72820,22.0,0.008999,0.008999
4,0.001855,1.887,2.3364,6.0,-74.153415,40.939709,19.302780,74.29314,23.0,0.010352,0.010352
...,...,...,...,...,...,...,...,...,...,...,...
185,0.000030,1.058,1.7307,3.0,-75.412711,39.657466,6.575272,65.28861,82.0,0.021523,0.021523
186,0.000030,1.220,0.7056,4.0,-75.474891,39.667591,1.432910,79.16454,95.0,0.005000,0.005000
187,0.000030,2.549,1.9926,4.0,-75.467436,39.674032,2.100324,74.97064,90.0,0.008097,0.008097
188,0.002469,1.798,5.1759,4.0,-75.058665,39.410337,12.894100,65.33733,11.0,0.010897,0.010897


In [8]:
data.describe()

,SLOPE,LENGTHKM,AREASQKM,StreamOrde,Avg_Lon,Avg_Lat,Avg_Elev,RunoffC,MajLC,Roughness,roughness
count,190.000000,190.000000,190.000000,190.000000,190.000000,190.000000,190.000000,190.000000,190.000000,190.000000,190.000000
mean,0.002102,2.165700,3.597518,3.568421,-74.517125,40.540920,50.146972,62.674149,42.489474,0.053002,0.053002
std,0.003399,1.672483,3.754582,1.261163,0.335840,0.446373,46.289138,7.626443,28.381516,0.059420,0.059420
min,0.000030,0.056000,0.005400,1.000000,-75.483389,39.410337,0.461794,46.279140,11.000000,0.005000,0.005000
25%,0.000076,0.901500,1.045800,3.000000,-74.623243,40.252289,20.220930,57.559730,21.000000,0.010536,0.010536
50%,0.000819,1.722000,2.328300,3.000000,-74.506990,40.612651,32.543145,62.281175,23.000000,0.028802,0.028802
75%,0.002353,2.862000,4.625550,4.000000,-74.277958,40.902186,64.269288,67.605240,82.000000,0.065343,0.065343
max,0.021397,7.934000,21.563100,6.000000,-73.995933,41.100752,252.981900,87.515460,95.000000,0.200000,0.200000


In [9]:
data.dtypes

SLOPE         float64
LENGTHKM      float64
AREASQKM      float64
StreamOrde    float64
Avg_Lon       float64
Avg_Lat       float64
Avg_Elev      float64
RunoffC       float64
MajLC         float64
Roughness     float64
roughness     float64
dtype: object

In [10]:
data[(data.roughness < 0.1999) & (data.roughness > 0.0051)]
# data = data[]

,SLOPE,LENGTHKM,AREASQKM,StreamOrde,Avg_Lon,Avg_Lat,Avg_Elev,RunoffC,MajLC,Roughness,roughness
0,0.000612,5.019,8.6067,4.0,-74.102551,40.936324,20.043160,62.65450,22.0,0.027188,0.027188
1,0.007341,0.331,0.9576,6.0,-74.180779,40.922627,40.657850,77.03477,23.0,0.022162,0.022162
2,0.002241,1.352,1.0440,6.0,-74.221956,40.890790,48.535850,71.74138,23.0,0.010536,0.010536
3,0.000030,0.985,0.8874,4.0,-74.078219,40.902936,14.687020,67.72820,22.0,0.008999,0.008999
4,0.001855,1.887,2.3364,6.0,-74.153415,40.939709,19.302780,74.29314,23.0,0.010352,0.010352
...,...,...,...,...,...,...,...,...,...,...,...
184,0.000030,1.369,0.9297,2.0,-75.241227,39.440139,12.546090,56.87803,21.0,0.160806,0.160806
185,0.000030,1.058,1.7307,3.0,-75.412711,39.657466,6.575272,65.28861,82.0,0.021523,0.021523
187,0.000030,2.549,1.9926,4.0,-75.467436,39.674032,2.100324,74.97064,90.0,0.008097,0.008097
188,0.002469,1.798,5.1759,4.0,-75.058665,39.410337,12.894100,65.33733,11.0,0.010897,0.010897


In [12]:
# Where the roughness calibration results in 0.3 or 0.005, this is likely an error related to flows
# We drop these data
# data = data[data.roughness < 0.1999]
# data = data[data.roughness > 0.0051]

# Convert data types into forms usable for regression
data.MajLC = data.MajLC.astype(str)
data.StreamOrde = data.StreamOrde.astype(str)

X = data.copy()

X = X.drop(columns=['roughness','Roughness'])   # all data but roughness is the input
X = pd.get_dummies(data=X) # turns categorical data into a form that regression can interpret
X

,SLOPE,LENGTHKM,AREASQKM,Avg_Lon,Avg_Lat,Avg_Elev,RunoffC,StreamOrde_1.0,StreamOrde_2.0,StreamOrde_3.0,StreamOrde_4.0,StreamOrde_5.0,StreamOrde_6.0,MajLC_11.0,MajLC_21.0,MajLC_22.0,MajLC_23.0,MajLC_24.0,MajLC_31.0,MajLC_41.0,MajLC_81.0,MajLC_82.0,MajLC_90.0,MajLC_95.0
0,0.000612,5.019,8.6067,-74.102551,40.936324,20.043160,62.65450,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0.007341,0.331,0.9576,-74.180779,40.922627,40.657850,77.03477,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
2,0.002241,1.352,1.0440,-74.221956,40.890790,48.535850,71.74138,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
3,0.000030,0.985,0.8874,-74.078219,40.902936,14.687020,67.72820,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0.001855,1.887,2.3364,-74.153415,40.939709,19.302780,74.29314,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,0.000030,1.058,1.7307,-75.412711,39.657466,6.575272,65.28861,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
186,0.000030,1.220,0.7056,-75.474891,39.667591,1.432910,79.16454,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
187,0.000030,2.549,1.9926,-75.467436,39.674032,2.100324,74.97064,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
188,0.002469,1.798,5.1759,-75.058665,39.410337,12.894100,65.33733,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0


In [13]:
X.dtypes

SLOPE             float64
LENGTHKM          float64
AREASQKM          float64
Avg_Lon           float64
Avg_Lat           float64
Avg_Elev          float64
RunoffC           float64
StreamOrde_1.0      uint8
StreamOrde_2.0      uint8
StreamOrde_3.0      uint8
StreamOrde_4.0      uint8
StreamOrde_5.0      uint8
StreamOrde_6.0      uint8
MajLC_11.0          uint8
MajLC_21.0          uint8
MajLC_22.0          uint8
MajLC_23.0          uint8
MajLC_24.0          uint8
MajLC_31.0          uint8
MajLC_41.0          uint8
MajLC_81.0          uint8
MajLC_82.0          uint8
MajLC_90.0          uint8
MajLC_95.0          uint8
dtype: object

In [22]:
# These are the available data columns
# Stream_H_m COMID StreamOrde fmax Predict_Stream_H_m roughness RMSE Majority_Land_Use \
# Avg_Lon Avg_Lat Avg_Elev RunoffC SLOPE LENGTHKM AREASQKM

y = data.roughness # only roughness is the target
# X

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2) #, random_state=5

reg = LinearRegression().fit(X_train, y_train)
print('Train R² = ' + str(round(reg.score(X_train, y_train),6)))
print('Test  R² = ' + str(round(reg.score(X_test, y_test),6)))

reg = LinearRegression().fit(X, y)
print('All   R² = ' + str(round(reg.score(X, y),6)))

# Train R² = 0.415594
# Test  R² = -0.995022
# All   R² = 0.364082

x=0
for i in reg.coef_:
    print(f'{round(i,6)} * {X.columns[x]}')
#     if x == 0: print(" (Bias) ",end="")
#     if x > 0:  print(" * " + X.columns[x] + " ",end="")
#     print(" + ")
    x += 1

Train R² = 0.488642
Test  R² = 0.142677
All   R² = 0.463088
7.272146 * SLOPE
-0.006956 * LENGTHKM
0.002623 * AREASQKM
0.018902 * Avg_Lon
-0.040931 * Avg_Lat
5.9e-05 * Avg_Elev
0.000822 * RunoffC
0.083634 * StreamOrde_1.0
0.005407 * StreamOrde_2.0
-0.00652 * StreamOrde_3.0
-0.023835 * StreamOrde_4.0
-0.013174 * StreamOrde_5.0
-0.045511 * StreamOrde_6.0
-0.019636 * MajLC_11.0
0.019601 * MajLC_21.0
0.011454 * MajLC_22.0
-0.002252 * MajLC_23.0
0.015778 * MajLC_24.0
-0.055104 * MajLC_31.0
0.015214 * MajLC_41.0
0.016573 * MajLC_81.0
0.024808 * MajLC_82.0
0.007789 * MajLC_90.0
-0.034226 * MajLC_95.0


In [15]:
X2 = data.copy()
X2 = X2['StreamOrde']
X2 = pd.get_dummies(data=X2)
reg3 = LinearRegression().fit(X2, y)
print('All   R² = ' + str(round(reg3.score(X2, y),6)))

All   R² = 0.293536


In [23]:
# This was an experiment in normalizing the data. It yeilded the exact same results
X2 = X.apply(lambda x: (x-x.min()) / (x.max()-x.min()) , axis=0)
X2

y2 = y/y.max()
reg2 = LinearRegression().fit(X2, y2)
print('All   R² = ' + str(round(reg2.score(X2, y2),6)))

x=0
for i in reg2.coef_:
    print(f'{round(i,6)} * {X2.columns[x]}')
#     if x == 0: print(" (Bias) ",end="")
#     if x > 0:  print(" * " + X.columns[x] + " ",end="")
#     print(" + ")
    x += 1

All   R² = 0.463088
0.776922 * SLOPE
-0.273987 * LENGTHKM
0.282772 * AREASQKM
0.140578 * Avg_Lon
-0.345952 * Avg_Lat
0.074493 * Avg_Elev
0.169444 * RunoffC
0.41817 * StreamOrde_1.0
0.027033 * StreamOrde_2.0
-0.032599 * StreamOrde_3.0
-0.119177 * StreamOrde_4.0
-0.065872 * StreamOrde_5.0
-0.227554 * StreamOrde_6.0
-0.098182 * MajLC_11.0
0.098007 * MajLC_21.0
0.05727 * MajLC_22.0
-0.011261 * MajLC_23.0
0.07889 * MajLC_24.0
-0.275519 * MajLC_31.0
0.076071 * MajLC_41.0
0.082867 * MajLC_81.0
0.12404 * MajLC_82.0
0.038947 * MajLC_90.0
-0.171131 * MajLC_95.0


In [20]:
x=0
for i in reg2.coef_:
    print(f'{round(i,6)} * {X.columns[x]} +')
#     if x == 0: print(" (Bias) ",end="")
#     if x > 0:  print(" * " + X.columns[x] + " ",end="")
#     print(" + ")
    x += 1

0.776922 * SLOPE +
-0.273987 * LENGTHKM +
0.282772 * AREASQKM +
0.140578 * Avg_Lon +
-0.345952 * Avg_Lat +
0.074493 * Avg_Elev +
0.169444 * RunoffC +
0.41817 * StreamOrde_1.0 +
0.027033 * StreamOrde_2.0 +
-0.032599 * StreamOrde_3.0 +
-0.119177 * StreamOrde_4.0 +
-0.065872 * StreamOrde_5.0 +
-0.227554 * StreamOrde_6.0 +
-0.098182 * MajLC_11.0 +
0.098007 * MajLC_21.0 +
0.05727 * MajLC_22.0 +
-0.011261 * MajLC_23.0 +
0.07889 * MajLC_24.0 +
-0.275519 * MajLC_31.0 +
0.076071 * MajLC_41.0 +
0.082867 * MajLC_81.0 +
0.12404 * MajLC_82.0 +
0.038947 * MajLC_90.0 +
-0.171131 * MajLC_95.0 +


In [55]:
X

,SLOPE,LENGTHKM,AREASQKM,Avg_Lon,Avg_Lat,Avg_Elev,RunoffC,StreamOrde_1.0,StreamOrde_2.0,StreamOrde_3.0,StreamOrde_4.0,StreamOrde_5.0,StreamOrde_6.0,MajLC_11.0,MajLC_21.0,MajLC_22.0,MajLC_23.0,MajLC_24.0,MajLC_31.0,MajLC_41.0,MajLC_81.0,MajLC_82.0,MajLC_90.0,MajLC_95.0
0,0.000612,5.019,8.6067,-74.102551,40.936324,20.043160,62.65450,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0.007341,0.331,0.9576,-74.180779,40.922627,40.657850,77.03477,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
2,0.002241,1.352,1.0440,-74.221956,40.890790,48.535850,71.74138,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
3,0.000030,0.985,0.8874,-74.078219,40.902936,14.687020,67.72820,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0.001855,1.887,2.3364,-74.153415,40.939709,19.302780,74.29314,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,0.000030,1.058,1.7307,-75.412711,39.657466,6.575272,65.28861,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
186,0.000030,1.220,0.7056,-75.474891,39.667591,1.432910,79.16454,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
187,0.000030,2.549,1.9926,-75.467436,39.674032,2.100324,74.97064,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
188,0.002469,1.798,5.1759,-75.058665,39.410337,12.894100,65.33733,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0


In [59]:
catch_data.describe()

,CatchId,SLOPE,LENGTHKM,AREASQKM,StreamOrde,Avg_Lon,Avg_Lat,Avg_Elev,RunoffC,MajLC
count,2.885100e+04,28851.000000,28851.000000,28851.000000,28851.000000,28851.000000,28851.000000,28851.000000,28851.000000,28851.000000
mean,1.776498e+07,0.010483,1.379283,2.124983,1.848601,-74.837122,40.587212,155.234552,59.284883,52.066722
std,1.013872e+08,0.020080,1.573990,3.792316,1.172928,0.627854,1.017216,170.953542,16.231988,27.274684
min,1.748535e+06,0.000030,0.001000,0.000900,1.000000,-76.384173,36.823690,-1.522519,40.000000,11.000000
25%,4.187324e+06,0.000030,0.275000,0.163800,1.000000,-75.355282,39.770167,13.221740,45.687160,41.000000
50%,6.202688e+06,0.002634,0.822000,0.882000,1.000000,-74.862570,40.655778,96.806100,54.385780,41.000000
75%,9.436410e+06,0.012356,1.996000,2.601450,2.000000,-74.340147,41.350020,248.135550,69.142790,82.000000
max,9.320404e+08,0.587143,21.558000,110.801700,6.000000,-73.190100,42.756209,971.405300,95.000000,95.000000


In [13]:
# This is the data we will use for regression
rdata = catch_data.copy()

# We are dropping the NaN data for simplicity at this time
rdata = rdata.fillna(-1)
rdata = rdata[rdata != -1]
rdata = rdata.dropna()
rdata = rdata[rdata.StreamOrde != -9]
# hucdata[hucdata.isna().any(axis=1)]
# hucdata[hucdata.isnull().any(axis=1)]

# Not all categorical data made it into the regression. Substitude unrepresented values with similars
rdata.loc[rdata.MajLC == 42, 'MajLC'] = 41
rdata.loc[rdata.MajLC == 43, 'MajLC'] = 41
rdata.loc[rdata.MajLC == 52, 'MajLC'] = 81
rdata.loc[rdata.MajLC == 71, 'MajLC'] = 81
rdata.loc[rdata.StreamOrde == 7, 'StreamOrde'] = 6

# Turn categories into strings. get_dummies turns categorical data into a form that regression can interpret
rdata.MajLC = rdata.MajLC.astype(str)
rdata.StreamOrde = rdata.StreamOrde.astype(str)
rdata = pd.get_dummies(data=rdata)
# rdata.head(2)

# Create the input for the regression. Must match the columns of regression data
rdata_in = rdata.drop(columns=['CatchId',])
predict = reg.predict(rdata_in)

# Write predictions to dataframe. Fit to bounds
rdata['Roughness'] = ''
rdata['Roughness'] = predict
rdata.loc[rdata['Roughness'] < 0.005, 'Roughness'] = 0.005
rdata.loc[rdata['Roughness'] > 0.300, 'Roughness'] = 0.200

# Create a df with just catchment and roughness. Merge back on to original rating curve table.
merge = rdata[['CatchId','Roughness']]
catch_data = catch_data.merge(merge, on='CatchId',how='left')

# Fill roughness values where regression data was not available.
catch_data.Roughness = catch_data.Roughness.fillna(0.05)

In [14]:
catch_data

,CatchId,SLOPE,LENGTHKM,AREASQKM,StreamOrde,Avg_Lon,Avg_Lat,Avg_Elev,RunoffC,MajLC,Roughness
0,932020122,0.049157,4.754,6.0120,1.0,-75.185438,43.127256,205.3807,NaN,NaN,0.050000
1,3246072,0.004730,2.260,1.6632,4.0,-74.565526,42.681145,371.1816,45.21915,81.0,0.005000
2,3246388,0.000030,0.168,0.0378,3.0,-74.156619,42.612042,285.9409,43.21429,81.0,0.005000
3,3247620,0.005577,0.624,1.0278,1.0,-74.116866,42.232266,716.3624,45.54291,43.0,0.142483
4,3247570,0.009604,21.108,80.7237,1.0,-74.214211,42.256132,650.3924,41.60782,41.0,0.228417
...,...,...,...,...,...,...,...,...,...,...,...
43279,10466733,0.000030,0.336,0.1845,NaN,NaN,NaN,NaN,NaN,NaN,0.050000
43280,10466483,0.000030,0.378,0.3438,NaN,NaN,NaN,NaN,NaN,NaN,0.050000
43281,10469587,0.002440,1.168,0.9054,NaN,NaN,NaN,NaN,NaN,NaN,0.050000
43282,10469373,0.000551,0.363,0.3555,NaN,NaN,NaN,NaN,NaN,NaN,0.050000


In [16]:
for catch in rough_data.COMID.unique():
    subset = rough_data[rough_data['COMID'] == catch]
    # local_rough = opt['roughness'].loc[opt['COMID'] == catch][0]
    local_rough = subset['roughness'].values[0]
    catch_data.loc[catch_data['CatchId'] == catch, 'Roughness'] = local_rough
    print('Rough at ' + str(catch) + ' is ' + str(local_rough))

# # Calculate discharge with the new regression-determined roughess
# hucdata_full['Discharge (m3s-1)'] = hucdata_full['WetArea (m2)'] * pow(hucdata_full['HydraulicRadius (m)'],2.0/3) * \
#         pow(hucdata_full['SLOPE'],0.5) / hucdata_full['Roughness2']

# hucdata_full.to_csv(path[:-4] + '-calc.csv',index=False)

Rough at 9513852.0 is 0.005874257550964357
Rough at 6246206.0 is 0.0050000100000000035
Rough at 6251128.0 is 0.008737403574680244
Rough at 6251234.0 is 0.0050000100000000035
Rough at 6251308.0 is 0.0050000100000000035
Rough at 6249784.0 is 0.0050000100000000035
Rough at 6251122.0 is 0.0050000100000000035
Rough at 9513850.0 is 0.19999999
Rough at 9513352.0 is 0.0903287364103574
Rough at 9513156.0 is 0.19999999
Rough at 6250358.0 is 0.0050000100000000035
Rough at 6250388.0 is 0.0050000100000000035
Rough at 9517822.0 is 0.012516531986623202
Rough at 9513466.0 is 0.023874554436640667
Rough at 6251144.0 is 0.0050000100000000035
Rough at 4481723.0 is 0.19999999
Rough at 9514000.0 is 0.0050000100000000035
Rough at 9514006.0 is 0.0050000100000000035
Rough at 6250622.0 is 0.019264772705746967
Rough at 9513346.0 is 0.19999999
Rough at 2590137.0 is 0.034001229236001705
Rough at 6249800.0 is 0.19999999
Rough at 6246098.0 is 0.04211842524584685
Rough at 9485856.0 is 0.010897114111348546
Rough at 44

In [18]:
catch_data.to_csv(catch_file)

In [24]:
# hucs = ['020301'] #['020200', '020301','020401','020402','020403']
# for huc in hucs:
huc = '020301'
huc_path = 'H:/My Drive/' + huc + '/hydrogeo-fulltable-' + huc + '.csv'
hucdata = pd.read_csv(path)
hucdata = hucdata.merge(catch_data[['CatchId', 'Roughness']], on='CatchId')

hucdata.loc[hucdata.CatchId == 6251308, 'SLOPE'] = 0.15 / 5223 # PASS1
hucdata.loc[hucdata.CatchId == 6251234, 'SLOPE'] = 0.36 / 6934 # PASS2 
hucdata.loc[hucdata.CatchId == 6251128, 'SLOPE'] = 1.27 / 1729 # PASS3
hucdata.loc[hucdata.CatchId == 6249784, 'SLOPE'] = 0.00005 # PASS7 - arbitrary because slope is almost zero
hucdata.loc[hucdata.CatchId == 6251122, 'SLOPE'] = 0.37 / (694 + 701) # PASS10
hucdata.loc[hucdata.CatchId == 6251144, 'SLOPE'] = 0.37 / (694 + 701) # PASS10
hucdata.loc[hucdata.CatchId == 6250358, 'SLOPE'] = 0.16 / 2744 # PASS16
hucdata.loc[hucdata.CatchId == 6250388, 'SLOPE'] = 0.71 / 3277 # PASS17
hucdata.loc[hucdata.CatchId == 6250418, 'SLOPE'] = 1.40 / 2824 # PASS18

hucdata.loc[hucdata.CatchId == 6246206, 'SLOPE'] = 1.59 / 2824 # WANQ2
hucdata.loc[hucdata.CatchId == 6250622, 'SLOPE'] = 1.41 / 2480 # POMP4/5
hucdata.loc[hucdata.CatchId == 6249738, 'SLOPE'] = 0.00005     # POMP2 - arbitrary value to remove error
# catch_data[catch_data.CatchId == 6246098, 'SLOPE'] = 0.88 / 1013 # RAMA5 - 
hucdata.loc[hucdata.CatchId == 932030072, 'SLOPE']=1.75 / 635  # RAMA5
hucdata.loc[hucdata.CatchId == 6250680, 'SLOPE'] = 0.00005     # ROCK2 - arbitrary value to remove error

hucdata.loc[hucdata.SLOPE < 0.00003, 'SLOPE'] = 0.00003

hucdata['Discharge (m3s-1)'] = hucdata['WetArea (m2)'] * pow(hucdata['HydraulicRadius (m)'],2.0/3) * \
    pow(hucdata['SLOPE'],0.5) / hucdata['Roughness_y']
hucdata

,CatchId,Stage,Number of Cells,SurfaceArea (m2),BedArea (m2),Volume (m3),SLOPE,LENGTHKM,AREASQKM,Roughness_x,TopWidth (m),WettedPerimeter (m),WetArea (m2),HydraulicRadius (m),Discharge (m3s-1),Roughness_y
0,6227452,0.0000,133,10604.098582,10717.675041,0.000000,0.123987,1.264,1.2339,0.05,8.389318,8.479173,0.000000,0.000000,0.000000,0.200000
1,6227452,0.3048,149,11879.771715,11993.653778,3422.739011,0.123987,1.264,1.2339,0.05,9.398554,9.488650,2.707863,0.285379,8.265965,0.200000
2,6227452,0.6096,166,13235.164248,13350.933168,7253.803373,0.123987,1.264,1.2339,0.05,10.470858,10.562447,5.738768,0.543318,26.909573,0.200000
3,6227452,0.9144,193,15387.841052,15510.719239,11599.104095,0.123987,1.264,1.2339,0.05,12.173925,12.271139,9.176506,0.747812,53.242411,0.200000
4,6227452,1.2192,217,17301.314961,17434.150253,16503.575210,0.123987,1.264,1.2339,0.05,13.687749,13.792840,13.056626,0.946623,88.647766,0.200000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427030,9515810,23.7744,401,32408.915906,32429.642733,720367.243751,0.012250,0.040,0.0324,0.05,810.222898,810.741068,18009.181094,22.213234,315016.426119,0.156602
427031,9515810,24.0792,401,32408.915906,32429.642733,730245.481319,0.012250,0.040,0.0324,0.05,810.222898,810.741068,18256.137033,22.517839,322248.869919,0.156602
427032,9515810,24.3840,401,32408.915906,32429.642733,740123.718887,0.012250,0.040,0.0324,0.05,810.222898,810.741068,18503.092972,22.822444,329546.833587,0.156602
427033,9515810,24.6888,401,32408.915906,32429.642733,750001.956455,0.012250,0.040,0.0324,0.05,810.222898,810.741068,18750.048911,23.127050,336910.024313,0.156602


In [28]:
hucdata.describe()

,CatchId,Stage,Number of Cells,SurfaceArea (m2),BedArea (m2),Volume (m3),SLOPE,LENGTHKM,AREASQKM,Roughness_x,TopWidth (m),WettedPerimeter (m),WetArea (m2),HydraulicRadius (m),Discharge (m3s-1),Roughness_y
count,4.270350e+05,427035.000000,4.270350e+05,4.270350e+05,4.270350e+05,4.270350e+05,427035.000000,427035.000000,427035.000000,4.270350e+05,427035.000000,427035.000000,4.270350e+05,4.270350e+05,4.269490e+05,427035.000000
mean,2.598171e+07,12.496800,9.980393e+03,8.023591e+05,8.051284e+05,7.925560e+06,0.010884,1.283783,1.807189,5.000000e-02,565.226511,567.507404,5.598492e+03,8.347262e+00,1.471720e+04,0.124351
std,1.290133e+08,7.302498,2.328656e+04,1.874033e+06,1.877281e+06,2.235934e+07,0.019130,1.471442,3.503150,3.991742e-13,1060.764795,1063.265829,1.277307e+04,5.493273e+00,3.624973e+04,0.074438
min,6.224162e+06,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,-2.549000e-02,0.000030,0.001000,0.000900,5.000000e-02,0.000000,0.000000,-1.460745e-05,-2.059456e-07,0.000000e+00,0.005000
25%,6.230326e+06,6.096000,9.000000e+02,7.206354e+04,7.251397e+04,3.398300e+05,0.000030,0.283000,0.189000,5.000000e-02,183.278454,184.562468,8.544391e+02,3.773534e+00,7.624412e+02,0.051613
50%,6.249600e+06,12.496800,3.160000e+03,2.529181e+05,2.545474e+05,1.729128e+06,0.003412,0.787000,0.757800,5.000000e-02,349.291533,351.413160,2.820934e+03,7.789357e+00,4.428759e+03,0.126042
75%,9.512728e+06,18.897600,1.018400e+04,8.170477e+05,8.207721e+05,6.782879e+06,0.013407,1.839000,2.212200,5.000000e-02,651.561554,654.472789,6.553879e+03,1.218292e+01,1.583540e+04,0.198820
max,9.320302e+08,24.993600,1.124623e+06,9.041069e+07,9.049700e+07,1.201770e+09,0.350000,24.645000,110.801700,5.000000e-02,54200.133817,54344.038679,1.069615e+06,2.499360e+01,1.222899e+06,0.299306


In [27]:
hucdata.to_csv(huc_path[:-4] + '_regr.csv')